In [ ]:
# utilize either training set to predict whether car price < 18,400 = 1 or 0 otherwise

# create the  binary variable

# use available features to predict the binary var

# only KNN

# can use any features included (year, mileage) or others that can be constructed (not from price var tho)



In [ ]:
# Question 2) 
# using the training data (large or small), draw a graph that replicates the pattern in 2.17

# Show that KNN estimation has following pattern:
# 1) training error decreases as 1/K increases
# 2) test error first decreases and then increases as 1/K (flexibility) increases

# Report code and graph

In [ ]:
# Question 3)

# Using training data only, draw a figure with:
# 1 and 2 types of error rates as a function of classification threshold
# 3) the overall error rate as a function of the classification threshold
# Replicate figure 4.7 for training set

# Draw an additional figure of the ROC curve (figure 4.8)